# Introduction and Purpose
The primary purpose of this notebook and set of methods/functions is to allow the DFCX script writer to copy/paste pages from one Flow to another Flow.  
In this notebook, we will demonstrate how to do this within the SAME agent, however these same methods/functions can be utilized to move pages BETWEEN agents!

## Imports
We're using the Legacy DFCX and DFFX libraries for this notebook.
Note that this scripting library is currently under a full refactor, so some of this code will slightly change in the future

In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from dfcx_scrapi.tools.copy_util import CopyUtil

In [ ]:
# Define your Service Account credentials file here, as well as the Agent ID you will be working on

creds = '<YOUR_CREDS_FILE>'
agent_id = '<YOUR_AGENT_ID>'

In [ ]:
# Instantiate your DFCX and DFFX Objects
cu = CopyUtil(creds)

## Get Flows Map from Agent in Reverse so we can reference by Flow Display Name
The DFFX Class has a set of `map` functions that are handy for manipulating CX Resources when you only need the UUID and Display Name, but not the entire object.
Here, we need to reference our Flows by Display Name, so we can use the `get_flows_map` method, and set the `reverse` flag to `True` giving us the Display Names as the keys and the UUIDs as the values.

In [ ]:
mfchat_flows_map = cu.flows.get_flows_map(agent_id, reverse=True)
mfchat_flows_map

## Get All Pages from Source Flow
Once we have our Flows map, we can reference it in our call to extract the full list of pages from the source Flow in question.
In our case, we want to extract pages from a Flow named `AMNT15`.  
We will call the `list_pages` method from the DFCX Class and pass in the appropriate dict reference.

In [ ]:
amnt15_pages = cu.pages.list_pages(mfchat_flows_map['AMNT15'])
print('AMNT15 Page Count = {}'.format(len(amnt15_pages)))

## Extract Subset of Pages that we want to copy
Now that we have our List of Page objects, we need to extract the subset of pages that we plan on copying over.
Usually you will do this using some regex matcher or pattern to select your pages.  
The easiest way to do this is to ensure the the Page designer has prepended the `page.display_name` with a specific label.
The more unique the matching pattern, the better!

Ex:
- TYS - Page 1
- TYS - Page 2
- TYS - Page N

In [ ]:
subset_pages = [] # define a list placeholder for your Page proto objects
for page in amnt15_pages:
    if 'TYS -' in page.display_name:
        subset_pages.append(page)

print('Total Pages to Copy = {}'.format(len(subset_pages)))

## Create Page Shells in Destination Flow
Using the `subset_pages` that we just collected, iter through them and create a "shell" page in the destination flow.  
This allows CX to assign a new UUID for the destination page which we will use to replace all references in the existing `subset_pages`

Remember to pass in the `Destination Flow ID` using the reverse flows map from above.

**NOTE** - If you have a lot of pages, consider a `time.sleep(.5)` in your loop so as to not overrun your API limits!

In [ ]:
for page in subset_pages:
    p.create_page(mfchat_flows_map['TYS'], display_name=page.display_name) # remember to add kwarg for display_name

## Prep Source Page Objects
In a previous step, we collected the `subset_pages` list of Page objects.  
Now we will use a ~!* MAGIC *!~ `CopyUtil` method called `convert_from_source_page_dependencies`

For `source` mode, you simply pass in the following:
- Agent ID String
- Original List of Page Objects (in our case, the `subset_pages` list) 
- STRING DISPLAY NAME of the Source Flow

The `source` mode changes all of the UUIDs in the Page object to be PLAIN TEXT STRING DISPLAY NAMES using the internal map functions.  
You will want to collect your new MODIFIED page objects in a new variable, in our case here we call it `subset_pages_prepped`

For `destination` mode, you simply pass in the following:
- Agent ID String
- MODIFIED page objects that you received as output from the `source` mode pass of this same function
- STRING DISPLAY NAME of the Destination Flow

The `destination` mode will perform a reverse dictionary lookup on all of the previously modified resources using the internal map functions.
You will want to collect your new MODIFIED FINAL page objects in a new variable, in our case here we call it `final_pages`

In [ ]:
subset_pages_prepped = cu.convert_from_source_page_dependencies(mfchat, subset_pages, 'AMNT15')

In [ ]:
final_pages = cu.convert_to_destination_page_dependencies(mfchat, subset_pages_prepped, 'TYS')

## Update Pages in Destination Flow
Loop through your `final_pages` list and call the `update_page` function

In [ ]:
for page in final_pages:
    cu.pages.update_page(page.name, page)
    print('Updated Page: {}'.format(page.display_name))

# Final Thoughts
All of the above could be improved by combining all of the functions into a single functions.
However, due to the shifting nature of bot building, I left the methods/functions separate to allow the script writer to make modifications or do sanity checks as they go along.

As noted at the start, this script could also be used to copy pages BETWEEN agents, and that is what it was originally written to do.